<a href="https://colab.research.google.com/github/bingxiaochen/ST-554-Project1/blob/main/Task1/ST_554_P1_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Bing Chen

Purpose: This programs is written for task 1 of project 1.

Date: 2-10-2026

# Read in data

To use the air quality data set available at the UCI machine learning repository, we want to first install the `ucimlrepo` library.

In [57]:
!pip install ucimlrepo

After installing it, import it in to use it. And, go ahead to import all other modules we might use.

In [58]:
import ucimlrepo as uci
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats


Now, we are ready to read in the air quality data from the library.

In [59]:
air_quality = uci.fetch_ucirepo(id=360)
air_quality.data.features

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


Now, let's do some data cleaning.

Let's check the type of each column.



In [60]:
air_quality.data.features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   int64  
 4   NMHC(GT)       9357 non-null   int64  
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   int64  
 7   NOx(GT)        9357 non-null   int64  
 8   PT08.S3(NOx)   9357 non-null   int64  
 9   NO2(GT)        9357 non-null   int64  
 10  PT08.S4(NO2)   9357 non-null   int64  
 11  PT08.S5(O3)    9357 non-null   int64  
 12  T              9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
dtypes: float64(5), int64(8), object(2)
memory usage: 1.1+ MB


In [61]:
air_quality.data.features.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,9.778305,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,10.900000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,17.200000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,24.100000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,44.600000,88.700000,2.231000


Now, we want to remove any observations where the `C6H6(GT)` or `CO(GT)` are -200 as these represent missing values. We will save the result as a new dataset. We can compare the shape of these datasets to see how many observations were removed.

In [62]:
air_clean = air_quality.data.features.loc[
    (air_quality.data.features["C6H6(GT)"] != -200) &
    (air_quality.data.features["CO(GT)"] != -200)
]

print(air_quality.data.features.shape)
print(air_clean.shape)


(9357, 15)
(7344, 15)


In [63]:
air_clean.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
count,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000,7344.000000
mean,2.129711,1110.580746,-150.224401,10.275735,947.198121,226.081563,826.920071,96.650327,1444.752723,1043.512936,17.770425,49.060076,0.989453
std,1.436472,218.681331,153.788325,7.441068,265.471608,227.144019,256.648430,85.089998,350.344170,405.569613,8.862688,17.451563,0.399889
min,0.100000,647.000000,-200.000000,0.200000,387.000000,-200.000000,322.000000,-200.000000,551.000000,221.000000,-1.900000,9.200000,0.184700
25%,1.100000,946.000000,-200.000000,4.600000,743.000000,90.000000,649.000000,73.000000,1203.000000,744.750000,11.200000,35.400000,0.698100
50%,1.800000,1075.000000,-200.000000,8.500000,919.000000,176.000000,795.000000,107.000000,1447.000000,990.000000,16.900000,49.300000,0.959700
75%,2.800000,1246.000000,-200.000000,14.300000,1125.250000,320.000000,960.000000,139.000000,1673.000000,1305.000000,23.800000,62.500000,1.258600
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,333.000000,2775.000000,2523.000000,44.600000,88.700000,2.180600


Looks like 2013 observations are removed.

Now, we are ready to make predictions of C6H6(GT).

# Prediction of C6H6(GT)

We want to be able to predict a value of `C6H6(GT)` for a new sample. We'll quantify the quality of the prediction through a loss function.


So, first, let's understand what a loss function is.

Suppose there is some prediction (call it $c$). The squared error loss for a data point (call it $y_1$) and $c$ is $L(y_1, c) = (y_1 − c)^2$

For a given set of data $(y_1,..., y_n)$, we could consider the mean squared error:

Root Mean Square Error = $RMSE(c) = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_1 − c)^2}$


As a prediction, we now want to choose the value of $c$ that minimizes this equation.



We'll use numerical methods to find the optimal value:

1. Grid search
2. Gradient descent

We'll implement these two algorithms for two equations representing $c$:

- we don’t consider any data other than  𝑦 , and c is just a constant that minimizes the function.
- we consider a linear equation (SLR model) of one other numeric variable.


## Grid Search

Grid search method follows the steps:  

1. Create a grid of values for $c$.
2. Find the RMSE for each value of $c$.
3. Determine which value of $c$ gives the optimal (smallest) RMSE.
4. And finally, report that as the prediction!


Now let's implement it together.



### Just y

We don't consider any data other than the $y$, then $c$ will just be some constant that minimises the function. To use a reasonable grid for $c$, let's consider using the first and third quartiles for the `C6H6(GT)`, since $c$ should live where most of the data are.


In [64]:
q1 = air_clean["C6H6(GT)"].quantile(0.25)
q3 = air_clean["C6H6(GT)"].quantile(0.75)

q1, q3

(np.float64(4.6), np.float64(14.3))

Now, create a grid of $c$ values using the quartiles. Let's create 1000 evenly spaced candidate values for $c$.

In [65]:
c_grid = np.linspace(q1, q3, 1000)

Next step is to find the RMSE for each value of c. Let's create a function to do that.

In [66]:
def rmse(y, c):
    """
    Find the RMSE for a given y and c.
    """
    y = np.asarray(y)
    return np.sqrt(np.mean([(yi - c) ** 2 for yi in y]))

Let's test out the function before moving to the next step. Pass the $c$ grid we created into the function.

In [67]:
rmse_values = [rmse(air_clean["C6H6(GT)"], c) for c in c_grid]
rmse_values

[np.float64(9.35820070736372),
 np.float64(9.352314968689592),
 np.float64(9.346435610689953),
 np.float64(9.340562645413627),
 np.float64(9.33469608492665),
 np.float64(9.328835941312219),
 np.float64(9.322982226670696),
 np.float64(9.317134953119549),
 np.float64(9.311294132793337),
 np.float64(9.305459777843682),
 np.float64(9.299631900439222),
 np.float64(9.293810512765596),
 np.float64(9.287995627025408),
 np.float64(9.282187255438194),
 np.float64(9.276385410240382),
 np.float64(9.270590103685272),
 np.float64(9.264801348042992),
 np.float64(9.259019155600475),
 np.float64(9.253243538661408),
 np.float64(9.247474509546212),
 np.float64(9.241712080591999),
 np.float64(9.235956264152541),
 np.float64(9.230207072598226),
 np.float64(9.224464518316028),
 np.float64(9.218728613709468),
 np.float64(9.212999371198572),
 np.float64(9.207276803219843),
 np.float64(9.201560922226209),
 np.float64(9.19585174068699),
 np.float64(9.19014927108786),
 np.float64(9.184453525930806),
 np.float64(

Good, looks like it's working correctly. Now, use a for loop to find which value of $c$ gives the smallest RMSE. We initialize with the first RMSE and its $c$, loop through the rest, every time we find a smaller RMSE, we update both values. Note that `rmse[i] corresponds to c_grid[i]`.

In [68]:
smallest_rmse = rmse_values[0]
best_c = c_grid[0]

for i in range(1, len(rmse_values)):
    if rmse_values[i] < smallest_rmse:
        smallest_rmse = rmse_values[i]
        best_c = c_grid[i]

smallest_rmse, best_c

(np.float64(7.440562418757215), np.float64(10.280180180180182))

There! $c = 10.280180180180182$ is our prediction!

Now, let's wrap all of that in a function.

In [69]:
def grid_search(y, c_grid):

    """
    Find the the optimal constant prediction for your response variable.

    Parameters
    ----------
    y : array-like
        The response variable.
    c_grid : array-like
        The grid of c values.
    """

    rmse_values = [rmse(y, c) for c in c_grid]

    smallest_rmse = rmse_values[0]
    best_c = c_grid[0]

    for i in range(1, len(rmse_values)):
        if rmse_values[i] < smallest_rmse:
            smallest_rmse = rmse_values[i]
            best_c = c_grid[i]

    return best_c, print(str(best_c) + " is your optimal constant prediction!")



Test our function with `C6H6(GT)` and `PT08.S1(CO)` variable.  

In [70]:
grid_search(air_clean["C6H6(GT)"], c_grid)

10.280180180180182 is your optimal constant prediction!


(np.float64(10.280180180180182), None)

The calculus based answer for this should comes out to be the sample mean. Let's check!

In [71]:
np.mean(air_clean["C6H6(GT)"])

np.float64(10.275735294117647)

Update our $c$ grid with values between Q1 and Q3 of `PT08.S1(CO)` variable in order to make a prediction on it.

In [74]:
c_grid_pt08 = np.linspace(air_clean["PT08.S1(CO)"].quantile(0.25),
                          air_clean["PT08.S1(CO)"].quantile(0.75), 1000)

In [75]:
grid_search(air_clean["PT08.S1(CO)"], c_grid_pt08)

1110.5645645645645 is your optimal constant prediction!


(np.float64(1110.5645645645645), None)

In [76]:
np.mean(air_clean["PT08.S1(CO)"])

np.float64(1110.5807461873637)

Great! Now let's implement grid search method using y and another numeric variable x.

### y and x
We'll use grid search to find the optimal
pair of values for $b_0$ (intercept) and $b_1$ (slope) using `PT08.S1(CO)` as our $x$ variable and `C6H6(GT)` as our $y$ variable. For each observation ($i$), our prediction is given by $c_i = b_0 + b_1x_i$. So, note that our grid now has two-dimensions.

This time, consider a grid of $b_0$ and $b_1$ values as follows:
- Use $b_0$ values from -25 to -15 with increments of 0.1.
- Use $b_1$ values from -5 to 5 with increments of 0.01


Let's create a function that takes in an x and and y column and outputs the optimal values (optimal $b_0$ and $b_1$ combination).

The RMSE formula is a bit different this time since $c_i = b_0 + b_1x_i$. So, the updated RMSE formula is: $RMSE(c) = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_i − (b_0 + b_1x_i)^2}$


Then we will use these values to predict a new `C6H6(GT)` for a `PT08.S1(CO)` of 946, 1075, and 1246.

In [77]:
def grid_search_SLR(y, x):

    """
    Find the the optimal intercept and slope for your linear equation of
    the response and explanatory variable.

    Parameters
    ----------
    y : array-like
        The response variable.
    x : array-like
        The explanatory variable.
    """


    y = np.asarray(y)
    x = np.asarray(x)

    b0_grid = np.arange(-25, -15, 0.1)
    b1_grid = np.arange(-5, 5, 0.01)

    rmse= [
    np.sqrt(np.mean((y - (b0 + b1 * x)) ** 2))
    for b0 in b0_grid
    for b1 in b1_grid
    ]

    # create coefficient pairs
    params = [(b0, b1) for b0 in b0_grid for b1 in b1_grid]

    # initialize the smallest rmse, best coefficients
    smallest_rmse = rmse[0]
    best_b0, best_b1 = params[0]

    # find best parameters
    for i in range(1, len(rmse)):
        if rmse[i] < smallest_rmse:
            smallest_rmse = rmse[i]
            best_b0, best_b1 = params[i]

    print(f"(b0, b1) = ({best_b0}, {best_b1}) is your optimal linear prediction!")

    return best_b0, best_b1, smallest_rmse



Now, let's test it!


In [78]:
grid_search_SLR(air_clean["C6H6(GT)"], air_clean["PT08.S1(CO)"])

(b0, b1) = (-22.99999999999997, 0.02999999999989278) is your optimal linear prediction!


(np.float64(-22.99999999999997),
 np.float64(0.02999999999989278),
 np.float64(3.5429053908306782))

Let's check our result with simple linear regression estimates from `SciPy.stats`.

In [79]:
x = air_clean["PT08.S1(CO)"].values
y = air_clean["C6H6(GT)"].values
results = stats.linregress(x, y)

results.intercept, results.slope

(np.float64(-22.955747380077504), np.float64(0.029922617322764876))

They are in close agreeement! Let's use the coefficient estimates to predict a new `C6H6(GT)` for a `PT08.S1(CO)` of 946, 1075, and 1246. Recall $c_i = b_0 + b_1x_i$.

In [87]:
# x = 946
-22.99999999999997+ 0.02999999999989278*946

5.3799999998985975

In [88]:
# x = 1075
-22.99999999999997+ 0.02999999999989278*1075

9.249999999884764

In [89]:
# x = 1246
-22.99999999999997+ 0.02999999999989278*1246

14.379999999866428

## Gradient Descent

The basic idea of gradient descent is to starts with an initial guess and then repeatedly adjusts that guess by moving in the negative direction of the function's slope. The size of each adjustment is controlled by a step size, and the process continues until the changes become very small or a maximum number of steps is reached. In this case, we use gradient descent to find the value of $c$ that minimizes the RMSE.

### Just y.

First, consider the case where we only use $y$ and no $x$. We can reuse the `rmse` function we creaeted earlier. Then, we need a function for the difference quotient (numerical derivative).

In [90]:
def diff_quotient(y, c, delta=0.001):
    return (rmse(y, c + delta) - rmse(y, c)) / delta

We want to use a while loop (we stop when movement < tolerance, or when `max_iters` is reached).


In [86]:
def gradient_descent_constant(y, start_c, step_size=0.01, num_tol=0.0001, delta=0.001, max_iters=10000):
    """
    Gradient descent to minimize RMSE and return the final c.

    Parameters
    ----------
    y : array-like
        Response variable.
    start_c : float
        Initial guess.
    step_size : float
        Step size.
    num_tol : float
        Numeric tolerance.
    delta : float
        Small change for difference quotient derivative approximation.
    max_iters : int
         safety that stops the loop after the maximum number of iterations is reached.
    """
    y = np.asarray(y)

    cur_c = float(start_c)
    iters = 0

    while iters < max_iters:
        slope = diff_quotient(y, cur_c, delta=delta)
        new_c = cur_c - slope * step_size

        if abs(new_c - cur_c) < num_tol:
            cur_c = new_c
            break

        cur_c = new_c
        iters += 1

    return cur_c